# Catnip

In [2]:
# Dependencies and configuration
import importlib
from src.config import settings, setup_dirs

izutsumiPaths, notIzutsumiPaths = setup_dirs()

## Pre-processing

### Panel extraction

In [ ]:
from modules.coreMPE.src.adenzu_panel.image_processing import panel

_ = panel.extract_panels_for_images_in_folder_recursive(
    input_dir=str(settings.paths.pages_dir),
    output_dir=str(settings.paths.panels_dir),
    split_joint_panels=False,   # maps to --split-joint-panels
    fallback=True              # maps to --fallback
)

### Head crops

In [ ]:
# Extract head crops (YOLOv8_animeface)
from src.headExtraction import anime_extraction_recursive

valid_exts = {".jpg", ".jpeg", ".png"}
panel_paths = sorted(
    [p for p in settings.paths.panels_dir.iterdir() if p.suffix.lower() in valid_exts]
)
num_crops = anime_extraction_recursive()

print(f"Extracted {num_crops} faces")

## Catnip core

### Model building

In [ ]:
import src.embeddingModel
importlib.reload(src.embeddingModel)

from src.embeddingModel import compute_embeddings, build_model, load_embeddings

#### New model

In [ ]:
# Build fresh model without loading weights (for new embedding computation)
embed_model = build_model(settings.img_size, settings.crops_dir, load_weights=False)
compute_embeddings(embed_model, settings.crops_dir, settings.img_size)

#### Previous model

In [ ]:
# Build embedding model (loads saved weights if available)
embed_model = build_model(settings.img_size, settings.crops_dir, load_weights=True)
embs, crop_paths = load_embeddings(settings.embed_path, settings.crop_path)

## Outputting

In [ ]:
import src.query
import src.output
importlib.reload(src.query) 
importlib.reload(src.output)

from src.query import izutsumi_query, izutsuminess_rank
from src.output import save_similar_results, char_nearest_neighbor

import matplotlib.pyplot as plt

In [ ]:
crop, index, score, thre = izutsumi_query(settings.embed_path,
                                        settings.crop_path,
                                        settings.img_size, 
                                        embed_model, 
                                        seed_paths,
                                        neg_paths,
                                        similarity_threshold=-1,
                                        alpha=0.5,
                                        mode='max')

In [ ]:
index = izutsuminess_rank(settings.embed_path, settings.crop_path, embed_model, seed_paths, neg_paths)

In [ ]:
plt.figure(figsize=(15, 5))
plt.bar(range(len(index)), score)
plt.xlabel("Izutsuminess")
plt.ylabel("Ranked Results")
plt.title("Embed score")
plt.show()

In [ ]:
cutoff = 0

results = char_nearest_neighbor(crop, index[cutoff:], score[cutoff:], thre)

In [ ]:
save_similar_results(crop, index, settings.output_dir, score)

## Finetune path

In [ ]:
import os, shutil, random
from pathlib import Path

out_dir = settings.training_dir

# create YOLO folder structure
for split in ["train", "val"]:
    for sub in ["images", "labels"]:
        (out_dir / split / sub).mkdir(parents=True, exist_ok=True)

# gather images

print(f"Izutsumi: {len(seed_paths)} | Not Izutsumi: {len(neg_paths)}")

# random split 80/20 for both classes
def split_data(imgs, val_ratio=0.2):
    random.shuffle(imgs)
    n_val = int(len(imgs) * val_ratio)
    return imgs[n_val:], imgs[:n_val]  # train, val

iz_train, iz_val = split_data(seed_paths)
not_train, not_val = split_data(neg_paths)

# copy images and create YOLO labels
def copy_and_label(imgs, split, class_id):
    for img in imgs:
        dest_img = out_dir / split / "images" / img.name
        shutil.copy(img, dest_img)
        # create empty label box (classification-style setup)
        label_path = out_dir / split / "labels" / (img.stem + ".txt")
        with open(label_path, "w") as f:
            f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")

copy_and_label(iz_train, "train", 1)
copy_and_label(iz_val, "val", 1)
copy_and_label(not_train, "train", 0)
copy_and_label(not_val, "val", 0)


In [ ]:
from ultralytics import YOLO, settings
from IPython.display import Image, display

# Load pretrained animeface model
model = YOLO("src/models/yolov8x6_animeface.pt")

# Fine-tune on your dataset
model.train(
    data="data/izutsumi_faces.yaml",
    epochs=50,
    imgsz=256,
    batch=16,
    lr0=1e-4,       # lower LR for finetuning
    freeze=10,      # freeze backbone layers
    project="runs/izutsumi_finetune",
    name="exp1"
)